# AUDIO SYNTHESIS WITH PYSNDLIB

This tutorial provides an introduction to working with Todd Ingalls' [pysndlib](https://pypi.org/project/pysndlib/), a python wrapper around [sndlib](https://ccrma.stanford.edu/software/snd/sndlib/), an audio synthesis library developed by Bill Schottstaedt at CCRMA, Stanford University.

To install pysndlib (mac and linux) do:

```
$ pip install pysndlib
```
<hr style="height:1px; color: grey;">

Notebook setup:

In [ ]:
import sys
sys.path.append("/Users/taube/Software/musx")
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Code
import math
import pysndlib.clm as clm
import pysndlib.sndlib as snd
import pysndlib.instruments as ins
import pysndlib.env as envs
import matplotlib.pyplot as plt
import musx    

def plotenv(env, title='', figsize=(3, 3)):
    xdata, ydata = env[::2], env[1::2]
    fig, ax = plt.subplots(figsize=figsize)
    if title: plt.title(title)
    ax.plot(xdata, ydata)
    plt.show()

def plotenvs(envs, titles=[], figsize=(6.4, 4.8), sharex=True, sharey=True):
    fig, axes = plt.subplots(nrows=1, ncols=len(envs), sharex=sharex, sharey=sharey, figsize=figsize)
    for i,env in enumerate(envs):
        xdata, ydata = env[::2], env[1::2]
        if titles: axes[i].title.set_text(titles[i])
        axes[i].plot(xdata,ydata)
    plt.show()
    
print(f"musx version: {musx.version}")

## Introduction

The pysndlib package provides support for reading and writing audio samples to files and <!--reading/writing audio samples to/from files--> and [numpy](https://pypi.org/project/numpy/) arrays.  The software consists of several large modules:

* pysndlib.clm : defines unit generators, utility functions, enum definitions, and input/output support for arrays and files, including the all-important `with Sound():` context.

* pysndlib.snd : defines functions and enums for manipulating audio file header data. sndlib supports over 70 different sound file formats; each format has its own notion of what header information consists of, and this module allows you to get and set header data in a fairly generic way.

* pysndlib.instruments : contains many historical audio instruments developed at CCRMA, including several of John Chowning's ground breaking FM instruments; reverb/spacialization tools; physical models; and various implementations of acoustic musical insturment using FM, amplitude modulation, additive syntheses, filters, etc.

* pysndlib.env : defines functions related to creating and modifying x,y envelopes.

This notebook imports these namespaces as 'clm', 'snd', 'ins', and 'envs' in an attempt to make it clear which pysndlib module a given reference belongs to. In a similar way, musx references always start with the 'musx' prefix.

## Building a simple audio instrument

In pysndlib an audio instrument is a function that outputs a stream of audio samples.  To generate the stream an instrument allocates unit generators (units of code that implement specific audio tasks) and runs them in a loop to output the stream of samples.  An audio sample is a number (int or float, depending) that represents the instantaneous amplitude of a waveform at a particular point in time. Each output sample is written to a particular location in a file; the exact location depends on the sampling rate -- the number of samples per second the audio player is processing.

The `simp()` instrument introduces these concepts along with several of the core unit generators and functions that pysndlib provides:

In [ ]:
def simp(start=0, dur=1, freq=440, amp=.5):
    beg = clm.seconds2samples(start)
    end = clm.seconds2samples(start + dur)
    osc = clm.make_oscil(freq)
    for i in range(beg, end):
        sample = clm.oscil(osc) * amp
        clm.outa(i, sample)

print(f"instrument: {simp}")

Here is a line-by-line explanation of the instrument.

1. Defines the simp instrument (function) with four input parameters:
    - start (*int | float*) : The start time of the sound in seconds, defaults to 0.
    - dur  (*int | float*) : The time that the sound lasts in seconds, defaults to 1 second.
    - freq (*int | float*) : The frequency of the sound in hertz, defaults to 440 hz.
    - amp (*int | float*) : The amplitude of the sound ranging from 0.0 to 1.0, defaults to .5.
2. Converts time in seconds into a starting sample position in the output audio stream.<!-- This position depends on the start time and the underlying sample rate: (`time` * SRATE), more about this later.-->
3. Same operation, but determines the ending sample position.
4. Allocates an `oscil()` unit generator to produce sine wave values (-1.0 to 1.0) at a given frequency (rate). 
5. Iterates `i` over all the sample positions from `beg` to `end`.
6. Gets the next sample returned by `oscil()` and scale it by the `amp` parameter.
7. Calls `outa()` to write the sample at the i'th position in channel a, where 'a' is an old-fashioned name for channel index 0.

## Generating audio files

Use pysndlib's `with Sound()` context to generate audio to a file. `with Sound()` behaves very much like Python's `with Open()` -- it provides a hygenic environment (safe read/write open/close) in which Python expressions can process data to files. It also provides [controls to customize the audio stream](https://testcase.github.io/pysndlib/with_sound.html); a number of these will will be used over the course of this notebook.

Here are some examples of generating audio files using `with Sound()`.

Call `simp()` using its default parameter values; play the file and print information as soon as the file is written:

In [ ]:
with clm.Sound("test.wav", play=True, statistics=True):
    simp()

Play the 4th and 5th octaves of the harmonic series as a musical scale with diminishing amplitudes:

In [ ]:
with clm.Sound("test.wav", play=True):
    scale = musx.harmonics(8, 32, fund=200.0)
    print(scale)
    for i,f in enumerate(scale):
        a = musx.rescale(i, 0, 32, 1, .01, mode='-exp')
        simp(start=i/2, dur=.4, freq=f, amp=a)

"Hello World!":

In [ ]:
with clm.Sound("test.wav", play=True):
    text, beg, rate = "Hello, World!", 0, .5
    for hz in [musx.hertz(ord(c)) for c in text]:
        simp(start=beg, dur=rate*.7, freq=hz/3, amp=.25)
        beg += rate

Sound cloud of 10 random pitches:

In [ ]:
with clm.Sound("test.wave", play=True):
    for r in range(10):
        simp(start=r/4, dur=8-(r/4), freq=musx.between(200, 600), amp=.1)

Chopsticks with left and right index fingers. There is no need to time sort the two independant hands: audio instruments write to absolute sample positions and a written sample will be merged with any sample value that is already there:

In [ ]:
with clm.Sound("test.wave", play=True):
    
    def chopsticks(hand, rate):
        right = "d5*8 d*8  f#*6 e f# g"
        left  = "c5*8 b4*8 a*8       g" 
        finger = musx.hertz(right if hand == 'right' else left)
        beg = 0
        for hz in finger:
            simp(start=beg, dur=rate*.75, freq=hz)
            beg += rate

    chopsticks("right", .25)
    chopsticks("left", .25)

## Adding audio controls

We can greatly improve on our `simp()` instrument by adding four new unit generators to the mix: `make_env()` and `env()` for envelope control, and `make_locsig()` and `locsig()` for placing audio at specific locations in speaker-space:

In [ ]:
def gimp(start=0, dur=1, freq=440, amp=.25, ampenv=[0, 0, .15, 1, .85, 1, 1, 0], 
         freqskw=0, freqenv=[0,0,1,0], degree=45, distance=0, reverb_amount=0):
    beg = clm.seconds2samples(start)
    end = clm.seconds2samples(start + dur)
    osc = clm.make_oscil(freq)
    ampenv = clm.make_env(ampenv, scaler=amp, duration=dur)
    glsenv = clm.make_env(freqenv, scaler=clm.hz2radians(freq) * freqskw, duration=dur)
    loc = clm.make_locsig(degree=degree, distance=distance, reverb=reverb_amount)
    for i in range(beg, end):
        samp = clm.env(ampenv) * clm.oscil(osc, clm.env(glsenv))
        clm.locsig(loc, i, samp)

print(f"instrument: {gimp}")

### Amplitude envelopes

In pysndlib an envelope is a Python list containing a series of x,y number pairs: [_x1,y1,x2,y2...xn,yn_]. The x values must be monotonically increasing left to right; the y values can range as they like. Envelopes are used to shape values for a certain effect. An envelope's coordinate system is not fixed, but it usually makes sense to work with "normalized" envelopes, e.g. envelopes who's x and y values range 0 -> 1, or 0 -> 100. This makes the envelope shape easy to grasp and you can scale and offset the envelope shape for different contexts.

In addition to adding envelope controls, `gimp()` provides several new parameters, including two that affect the amplitude and location of the sound:

* ampenv (*list*) : An x,y amplitude envelope, defaults to a three segment trapezoid (rise, steady-state, fall).
* degree (*int* | *float*) : The sound's location, in degrees, within a stereo speaker space.

This next example highlights the improvements implemented in `gimp()`. For example, the original `simp()` instrument produces notes that start and end with an annoying 'snap', and its audio signal appears in only the left channel of a two-channel audio file. The snap occurs because `simp()` moves to its maximum amplitude instantaneously, over the course of one sample. The audio appears only in the left channel because that is what the `outa()` generator is defined to do. In comparison, the amplitude envelope in `gimp()` ramps amplitude up and down over a number of samples, and its `locsig()` generator places audio in multichannel space:

In [ ]:
simpenv, gimpenv =  [0, 1, 1, 1], [0, 0, .15, 1, .85, 1, 1, 0]
plotenvs([simpenv, gimpenv], figsize=(3,1), titles=["simp() amp", "gimp() amp"])

with clm.Sound("test.wav", play=True, channels=2):
    simp(start=0, dur=1, amp=.9)
    gimp(start=2, dur=1, amp=.9)

In this example `gimp()`  generates eleven notes; each note receives its own unique amplitude shape as the initial normalized envelope morphs into its mirror image:

In [ ]:
ampenvs = [[0, 1-i, 1, 0+i] for i in musx.frange(0, 1.1, .1)]
plotenvs(ampenvs, titles=[i for i in range(1,12)], figsize=(8,1))

with clm.Sound("test.wav", play=True, channels=2):
    for i, env in enumerate(ampenvs):
        gimp(i, .75, 440, .9, ampenv=env)

Exponential amplitude envelopes work well for percussive sounds. This example plays a nice percussive 2 octave pentatonic scale.

In [ ]:
scale = musx.scale(60, 16, 2, 2, 3, 2, 3 )
ampenv = musx.exp_env()
print(f"scale: {scale}")
print(f"ampenv: {ampenv}")
plotenv(ampenv, title="perc env", figsize=(2,1))

with clm.Sound("test.wav", play=True, channels=2):
    for t, k in enumerate(scale):
        for u in musx.frange(0, .3, .1):
            gimp(t*.3+u, .15, musx.hertz(k), .9, ampenv=ampenv)

### Frequency Envelopes

Envelopes can be applied to frequency as well as amplitude. A frequency envelope is effectively a shaped glissando. The `gimp()` instrument has two parameters that control frequency changes:

* freqskw (*int* | *float*) : An amount to move (skew) from the `freq` parameter's value. Both positive and negative skews are possible. If skew is 0 then the frequency envelope will have no effect. The default freqskw value is 0.
* freqenv (*list*) : An x,y envelope; at maximum y the maximum frequency skew will be reached. Defaults to zero [0, 0, 1, 0] (no effect).

Line 7 and 8 in gimp()'s code create the amplitude and frequency envelopes. 

>ampenv = clm.make_env(ampenv, scaler=amp, duration=dur)

>glsenv = clm.make_env(freqenv, scaler=clm.hz2radians(freqskw), duration=dur)
    
Note that envelopes applied to frequency must be expressed in *radians*, not hertz or amplitudes!

To produce a frequency envelope with gimp() you must provide non-zero values for both the `freq` and the `freqskw` parameters:

In [ ]:
freqenv=[0, 0, .30, 0, .70, 1, 1, 1]
print(f"freqenv: {freqenv}")
plotenv(freqenv, title="freqenv", figsize=(2,1))

with clm.Sound("test.wav", play=True, channels=2):
    gimp(start=0, dur=2, freq=220, amp=.9, freqskw=3, freqenv=freqenv)

Skews can go negative:

In [ ]:
freqenv = [0, 0, .25, -1, .75, 1, 1, 0]
print(f"freqenv: {freqenv}") 
plotenv(freqenv, figsize=(2,1), title="skew down and up")

with clm.Sound("test.wav", play=True, channels=2):
    #for i, env in enumerate(ampenvs):
    gimp(start=0, dur=2, freq=440, amp=.9, freqskw=.5, freqenv=[0, 0, .25, -1, .75, 1, 1, 0])

Siren:

In [ ]:
freqenv = [0,0]
for i in range(1, 11):
    if i % 1 == 0: 
        freqenv.extend([(i/10)-.01, 1, i/10, 0])
print(f"freqenv: {freqenv}")    
plotenv(freqenv, figsize=(3,2))

with clm.Sound("test.wav", play=True, channels=2):
    gimp(start=0, dur=2.5, freq=440, amp=.9,  freqskw=1.5, freqenv=freqenv)

Six randomized bird calls ("Solitary Vireo", Bill Schottstaedt)

In [ ]:
ampenv = [0.0, 0.0, .25, 1.0, .60, .70, .75, 1.0, 1.0, 0.0]
freqenv = [0.0, .20, .03, .30, .06, .10, .10, .50, .13, .40, .16, .80, .19, .50, 
           .22, .90, .25, .60, .28, 1.0, .31, .60, .34, 1.0, .37, .50, .41, .90,
           .45, .40, .49, .80, .51, .40, .54, .75, .57, .35, .60, .70, .63, .30,
           .66, .60, .69, .25, .72, .50, .75, .20, .78, .30, .82, .10, .85, .30,
           .88, .05, .91, .30, .94, 0.0, .95, .30, .99, 0.0, 1.0, .10]
plotenvs([ampenv, freqenv] , figsize=(4,2), titles=["amp env", "freq env"])

with clm.Sound("test.wav", play=True, channels=2):
    for i in range(6):
        gimp(i, dur=musx.vary(.4, .2), freq=musx.vary(1800, .4) , amp=musx.vary(.75, .15) , ampenv=ampenv,
             freqskw=musx.vary(.5, .35), freqenv=freqenv)      

### Signal placement

Two channel audio provides 90 degrees of speaker space to work with: degree 0 is full left, 45 is center (equal power in both speakers) and 90  is full right. The range of 0-90 degrees is one 'segment' in a potential speaker space of 360 degrees:

| Left  | Center | Right |
| :---  | :----: |  ---: |    
| 0/360 |   45   |   90  |
| 315   |  you  |  135  |
| 270   |  225   |  180  |


In this next example a single sinewave controls both pitch and speaker location.  As the sinewave's amplitude moves between -1 and 1, the speaker location moves between 0 and 90 degrees and pitch ranges from 220 to 440 hertz. The instrument is using an exponential amplitude envelope to produce the rapid decay of a percusive instrument.

(This example will work best if you use headphones.)

In [ ]:
with clm.Sound("test.wav", play=True, channels=2):
    ampenv = musx.exp_env()
    rate = 1 # 2 3 7
    tops = [440, 880, 440, 770, 440, 660, 440, 550, 440]
    beg = 0
    for top in tops:
        location, frequency = [], []
        for time in musx.frange(0, 4.01, .1):
            sinewave = math.sin(2*math.pi*rate*time)
            degree = musx.rescale(sinewave, -1, 1, 0, 90)
            location.extend([time, degree])
            pitch = musx.rescale(sinewave, -1, 1, 220, top)
            frequency.extend([time, pitch])
            gimp(beg+time, .75, pitch, .75, degree=degree, ampenv=ampenv)
        if beg == 0:
            plotenvs([ampenv, location, frequency], titles=["amp env", "location", "frequency"],
                     figsize=(8,2), sharex=False, sharey=False)
        beg += 4.1 

### Reverberation

<!-- Reverberation occurs when a space with (reflecting) surfaces scatters sound in different directions and with different -->
 Reverberation occurs in an enclosed envronment when reflective surfaces scatter sound in different directions and at different phases, producing a multitude of diffused, stretched copies of the original sound.  In the digital domain this can be achieved in a variety ways, a common solution involves routing the audio signal through a combination of filters, e.g. networks of all-pass, comb, and low-pass filters, to achieve the desired effect.  Over the years composers and researchers at CCRMA developed a number of different reverberators (prcrev, kiprev, nrev, jcrev, jlrev, tankrev, converb...), of these, the `nrev()` reverberator has probably been the most used and is featured in the next few examples.

Adding reverberation to a pysndlib audio instrument involves several steps:

1. The instrument needs to include a unit generator such as `locsig()` to handle the reverberation.
2. Reverberation parameters are added to the instrument so users can control reverb quality on a note by note basis.
3. The `with Sound():` context must receive a reverberator instance to use with the instrument calls.
    
The `gimp()` instrument already includes two parameters that affect reverberation:

* distance : (*int* | *float*), A distance factor to use with reverberation. Distance has no units, but as its value increases the signal is perceived to be recededing further into the distance. Defaults to 1.  
* reverb_amount (*int* | *float*) : an amount of the signal to reverberate, e.g. 'reverb_amount=.001'. Take care with this value -- it is possible to "blow up" the reverberator with too high a value. Experiment with different settings carefully, start out listening to small quantites such as .001, or .01 and work your way up as necessary.  Defaults to 0.

In line 7 `gimp()`  passes  the distance and reverb_amount values to `locsig()` along with the degree setting:

> loc = clm.make_locsig(degree=degree, distance=distance, reverb=reverb_amount)   

In a DAW reverb is typicically applied to an entire track.  While this is also possible in pysndlib, the reverb in pysndlib can be specific to each *instrument*: one sound file can contain multiple instruments, each of which can apply unique locations, distances, and reverb amounts.  The default values for locsig are:

> reverb_factor=1.09, lp_coeff=.7, volume=1.0, decay_time=1.

##### Applying the `nrev()` reverberator

In the next example gimp plays 5 percussive notes with increasing amounts of reverb, the last note 'clips' because at that point the reverberator is oversaturated, as shown in this image.

<div>
   <center><img src="support/rev1.png" width="500"></center>
</div>

If you are using headphones check your levels before playing!

In [ ]:
ampenv = [0.0, 1.0, 0.1, 0.5, 0.2, 0.25, 0.3, 0.125, 0.4, 0.0625, 0.5, 0.03125, 
          0.6, 0.015625, 0.7, 0.0078125, 0.8, 0.00390625, 0.9, 0.001953125, 1.0, 0.0]

revamounts = [0, .001, .01, .1, .5, 1.5]

with clm.Sound("test.wav", play=True, channels=2, reverb=ins.nrev(),reverb_channels=1):
    for t, revamt in enumerate(revamounts):
        print("rev_amount:", revamt)
        gimp(t, .1, 440, .8, ampenv, degree=45, distance=1, reverb_amount=revamt)

This example reduces the previous reverb amount to .5 so the amplitude no longer clips. However, the audio still clips on the x axis because the file stops at the time+duration of the last note, which is earlier than the time at which the heavy reverberation tail can reach zero:
<!-- ![rev2.png](attachment:ceaf3980-c7cc-4226-ab42-1af593415231.png) -->

<div>
   <center><img src="support/rev2.png" width="500"></center>
</div>

In [ ]:
revamounts = [0, .001, .01, .1, .5]

with clm.Sound("test.wav", play=True, channels=2, reverb=ins.nrev(),reverb_channels=1):
    for t, revamt in enumerate(revamounts):
        print("rev_amount:", revamt)
        gimp(t, .1, 440, .8, ampenv, degree=45, distance=1, reverb_amount=revamt)

#### nrev parameters

To fix the x-axis clipping, specify a `decay_time` to nrev when it is passed to the context.

The nrev reverberator has four parameters:

* reverb_factor _(float)_ : controls the length of the reverb decay, defaults to 1.09 and should not exceed 1.215.
* lp_coeff _(float)_ : controls the strength of the low pass filter inserted in the feedback loop, defaults to .7.
* volume _(float)_ : amplitude scaler on the reverb output, defaults to 1.0.
* decay_time _(int | float)_: amount of time for the reverb's tail to drain to 0. Defaults to 1 second.

The default values for the first three prameters work well in most cases but the `decay_time` value is just a guess and won't work well for endings with heavy reverb.

In this example nrev's `decay_time` is set to 2.5 seconds, long enough for the reverb tail to disappear even if headphones are on:

In [ ]:
with clm.Sound("test.wav", play=True, channels=2, reverb=ins.nrev(decay_time=2.5),reverb_channels=1):
    for t, revamt in enumerate(revamounts):
        print("rev_amount:", revamt)
        gimp(t, .1, 440, .8, ampenv, degree=45, distance=1, reverb_amount=revamt)

This example adds reverb, distance and degree to every other chord in the appeggiated chords routine. Degrees are applied to each note, distance and reverb are applied to every other chord:

In [ ]:
with clm.Sound("test.wav", play=True, channels=2, reverb=ins.nrev(decay_time=2.5),reverb_channels=1):
    ampenv = musx.exp_env()
    rate = 1 #2 3 4
    tops = [440, 880, 440, 770, 440, 660, 440, 550, 440]
    beg = 0
    for num, top in enumerate(tops, 1):
        dist, rev = (num, .003 * num) if num % 2 == 0 else (1, 0)
        for time in musx.frange(0, 4.01, .1):
            sinewave = math.sin(2 * math.pi * rate * time)
            degree = musx.rescale(sinewave, -1, 1, 0, 90)
            pitch = musx.rescale(sinewave, -1, 1, 220, top)
            gimp(beg + time, .75, pitch, .75, degree=degree, ampenv=ampenv, reverb_amount=rev, distance=dist)
        beg += 4.1

## CLM instruments

Designing interesting audio instruments is as much an art form as creating interesting compositions that use them! Over many years CCRMA produced a whole host instruments and compositions that have had both musical and commercial success. Many of these instruments are offshoots from John Chowning's seminal paper <a href="https://ccrma.stanford.edu/sites/default/files/user/jc/fm_synthesis_paper.pdf">The Synthesis of Complex Audio Spectra by Means of Frequency Modulation</a>.  Chowning's paper eventually resulted in the production of Yamaha's DX synthesizers, the most successful synth series in the history of electronic music. Years later Julius Smith produced his <a href="https://ccrma.stanford.edu/~jos/pasp/">Physical Model</a> approach at CCRMA, which became another (entirely different) commercially successful technique.

This next instrument models the FM algorithm described in Chowning's article (pp. 6-10) using the nomenclature of the paper so you can match the code to the descriptions in the paper. The article's instrument parameters are:

* P1 = Begin time of instrument
* P2 = Instrument number (IGNORED)
* P3 = Duration of the "note"
* P4 = Amplitude of the output wave
* P5 = Carrier frequency
* P6 = Modulating frequency
* P7 = Modulation index 1, I1
* P8 = Modulation index 2, I2

In the paper Chowning refers to amplitude and timbre envelopes that are not represented by instrument parameters. These are added as `amp_env` and `index_env` arguments to the instrument:

In [ ]:
def proto_fm(P1, P3, P4, P5, P6, P7, P8, amp_env, index_env):
    start = clm.seconds2samples(P1)
    end = clm.seconds2samples(P1 + P3)
    amp_env = clm.make_env(amp_env, scaler=P4, duration=P3)
    carrier = clm.make_oscil(P5)
    modulator = clm.make_oscil(P6)
    dev1 = P7 * P6
    dev2 = (P8 - P7) * P6
    dev_env = clm.make_env(index_env, offset=clm.hz2radians(dev1), scaler=clm.hz2radians(dev2), duration=P3)
    for i in range(start, end):
        deviation = clm.env(dev_env) * clm.oscil(modulator)
        clm.outa(i, clm.env(amp_env) * clm.oscil(carrier, deviation))
     
print(f"instrument: {proto_fm}")

#### Audio examples from the paper

Generic brass-like tones:

In [ ]:
with clm.Sound("test.wav", play=True):
    #amp_env = [0, 0, 1/8, 1, 2/6, .7, 5/6, .6, 1, 0]
    amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
    index_env = amp_env
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=.6, P4=.5, P5=440, P6=440, P7=0, P8=5, amp_env=amp_env, index_env=index_env)

Generic woodwind-like tone:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env = [0, 0, .075, 1, 2/6, .7, 5/6, .6, 1, 0]
    index_env = [0, 0, .075, 1,  .1, 1, 1, .95]
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=.6, P4=.5, P5=900, P6=300, P7=0, P8=2, amp_env=amp_env, index_env=index_env)

Bassoon-like tone:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env = [0, 0, .075, 1, 2/6, .7, 5/6, .6, 1, 0]
    index_env = [0, 0, .075, 1,  .1, 1, 1, .95]
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=.6, P4=.5, P5=500, P6=100, P7=0, P8=1.5, amp_env=amp_env, index_env=index_env)

Clarinet-like tone:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env = [0, 0, .075, 1, 2/6, .7, 5/6, .6, 1, 0]
    ind_env = [0, 0, .075, 1,  .1, 1, 1, .95]
    plotenvs([amp_env, ind_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=.6, P4=.5, P5=900, P6=600, P7=4, P8=2, amp_env=amp_env, index_env=index_env)

Bell-like tones:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env = musx.exp_env(20, 2)
    index_env = amp_env
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=8, P4=.8, P5=200, P6=280, P7=0, P8=10, amp_env=amp_env, index_env=index_env)

Drum-like tone:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env = musx.exp_env(10, 3)    
    index_env = [0, .75, .2, 1, .4, .25, .6, .1, 1, 0]
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=.2, P4=.9, P5=200, P6=280, P7=0, P8=2, amp_env=amp_env, index_env=index_env)

Wood drum -like tone:

In [ ]:
with clm.Sound("test.wav", play=True, channels=2):
    amp_env = musx.exp_env(10, 3)    
    index_env = [0, 1, .025, 0, 1, 0]
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    proto_fm(P1=0, P3=1, P4=.9, P5=80, P6=55, P7=0, P8=25, amp_env=amp_env, index_env=index_env)

#### A Simple FM Instrument

Here is a rewrite of the prototype instrument that uses descriptive parameter names and adds several improvements. It is called "simple fm" because the instrument uses only two oscillators,  as opposed to more complex arrangements found in cascade fm and parallel fm:

In [ ]:
def simple_fm (start, duration, carrier, amplitude, mc_ratio=1, fm_index=1,
               amp_env=[0,0,.25,1,.75,1,1,0], amp_attack=None, amp_release=None,
               index_env=[0, 1, 1, 1], index_attack=None, index_release=None,
               degree=45, distance=1, reverb=0):
    start = clm.seconds2samples(start)
    end = start + clm.seconds2samples(duration)
    car_osc = clm.make_oscil(carrier)
    mod_osc = clm.make_oscil(carrier * mc_ratio)
    fm_index = clm.hz2radians(fm_index * mc_ratio * carrier)
    if amp_attack:
        amp_env = musx.segment_env(amp_env, duration, amp_attack, amp_release)
    if index_attack:
        index_env = musx.segment_env(index_env, duration, index_attack, index_release)
    amp_env = clm.make_env(amp_env, scaler=amplitude, duration=duration)
    index_env = clm.make_env(index_env, scaler=fm_index, duration=duration)
    for i in range(start, end):
        clm.outa(i, clm.env(amp_env) * clm.oscil(car_osc, clm.env(index_env) * clm.oscil(mod_osc)))
        
print(f'instrument: {simple_fm}')

The added improvements are:

* An `mc_ratio` (modulator-to-carrier ratio) parameter replaces frequencies in P5 and P6. In the new instrument the modulation frequency is calculated as `carrier*mc_ratio`, which makes the ratio relationship much clearer than using absolute hertz values.
* Uses the `locsig` unit generator to control degree, distance and reverb.
* Provides optional 'attack' and 'decay' envelope segments so that these amplitude durations are fixed regardless of how long the note lasts.  Without this, an envelope's attack and decay times will be proportional to the duration of the note (the longer the note duration the slower the envelope evolves), which is not how most instruments behave.

The next four examples illustrate how the attack (and to a lesser degree, the decay) of a sound are critical features for identifying what kind of instrument is playing. 

<!-- In the first tone the instrument has a constant amplitude and timbre so it is difficult to recognize the instrument at all.  The second example uses the original envelopes on amplitude and timbre so the tone sounds brass-like. The third example is like the second, but its duration is 10 times longer. Since envelopes are proportional to the duration, the attack and decay segments are stretched out so much that the tone does not sound much like a brass instrument any more. -->

If a brass-like tone has no attack or decay features it is difficult to recognize the instrument at all:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env=[0, 1, 1, 1]
    index_env=amp_env
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    simple_fm(0, .6, 440, .5, 1.0, 5.0, amp_env=[0,1, 1, 1], index_env=[0,1, 1, 1])

The original envelopes for amplitude and timbre work fine when the note is short:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
    index_env = amp_env
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    simple_fm(0, .6, 440, .5, 1.0, 5.0, amp_env=amp_env, index_env=index_env)

But if the note has a long duration (6 second in this example) the attack and decay segments will be so 'stretched out' that the tone loses it's brass qualities again:

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
    index_env = amp_env
    plotenvs([amp_env, index_env], titles=["amp_env", "index_env"], figsize=(4,1))
    simple_fm(0,  6, 440, .5, 1.0, 5.0, amp_env=amp_env, index_env=index_env)

Use the `amp_attack` and `index_attack` parameters to insure exact attack times regardless of how long the note lasts:

- amp_attack [ *attack_x* | *attack_dur* ] : A list holding the x coordinate that ends the amplitude envelope's attack portion and the attack's duration in seconds. 
- index_attack [ *attack_x* | *attack_dur* ] : A list holding the x coordinate that ends the index envelope's attack portion and the attack's duration in seconds. 

There are equivalent parameters for the (optional) decay times.

In this example attack values are [1/3, .1], which declares that the attack portion of the envelope ends at x-coordinate = 1/3 and the duration of the segment is .1 seconds; with these values in effect the 6 second note sounds brass-like again:

In [ ]:
with clm.Sound("test.wav", play=True):
    dur = 6
    amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
    attack_vals =          [1/3, .1]
    plotenvs([amp_env, musx.segment_env(amp_env, 6, attack_vals, attack_vals)], titles=["original", "adjusted"], figsize=(4,1))
    index_env=amp_env
    simple_fm(0, 6, 440, .5, 1.0, 5.0, amp_env=amp_env, index_env=index_env,
              amp_attack=attack_vals, index_attack=attack_vals)

Name this tune!

In [ ]:
with clm.Sound("test.wav", play=True):
    amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
    index_env=amp_env
    melody = musx.hertz( "g4 e c e g c5 e  d c e4 f# g g g e5 d c b4 a  b c5 c g4 e c")
    rhythm = musx.rhythm("e. s q*3   h  e. s q*3     h e*2 q. e q h  e. s q*5", tempo=80)
    t=0
    for m,r in zip(melody, rhythm):
        simple_fm(t, r, m, .5, fm_index=4.0, amp_env=amp_env, index_env=index_env,
                  amp_attack=[1/3, .1], index_attack=[1/3, .1])
        t += r

#### The pysndlib instrument collection

The pysndlib distribution contains many interesting audio instruments developed by composers and researchers at CCRMA over the last fifty years or so. These instruments use a variety of syntheses techniques and demonstrate most of the unit generators defined in pysndlib. While each digital instrument is unique, you will find that many are also 'flexible' -- capable of producing a variety interesting sounds other than the acoustic qualities of the instrument that they are named for.

Bill Shottaedt's `fm_violin()` is particularily robust and flexible audio instrument (and is also an example of "complex fm" using three modulators).


A good way to learn about audio synthesis is to work through pysndlib's catalogue of instruments, find instruments that interest you, then (1) experiment with parameter settings (2) study the code and (3) add or improve features in the instrument. 

This demonstrates two short examples of fm_violin() as a stringed instrument:

In [ ]:
with clm.Sound(play=True, reverb=ins.nrev(decay_time=2.5)):   # reverb=jc_reverb()
    print("Arco - gliss - pizz")
    arco =  [0, 0, 5, 1,   98, .6, 100, 0] # 50, 1,
    gliss = [0, 0, 40, 0, 50, 1, 100, 1]
    pizz  = [0.0, 1.0, 0.2, 0.5, 0.4, 0.25, 0.6, 0.125, 0.8, 0.0625, 1.0, 0.03125]
    rev = .025
    
    plotenvs([arco, gliss, pizz], titles=["arco", "gliss", "pizz"], figsize=(4,1))

    # Arco - gliss - pizz
    t = 0
    ins.fm_violin(t,    4,   220,     .4, fm_index=5, amp_env=arco, gliss_env=gliss, glissando_amount=3.1, reverb_amount=rev)
    t += 4
    ins.fm_violin(t+.05,     .25, 220*3/2, .9, fm_index=4, amp_env=pizz, reverb_amount=rev)
    ins.fm_violin(t+.06, .27, 220,     .9, fm_index=4, amp_env=pizz, reverb_amount=rev)

    # Mendelssohn violin concerto:
    print("Mendelssohn violin concerto")
    rev = .1
    t += 3
    mel = musx.hertz( "b5 b b g e6 e b5 g6 fs e c e b5 b c6 b5 a a a e6 b5 c6 b5 a a a ds6 b5")
    rhy = musx.rhythm("q. e h q q  h q  q  q*4      h. q q  e  e q q h. q  q  e  e q q h   h", tempo=160)
    for m,r in zip(mel, rhy):
        ins.fm_violin(t, musx.vary(r, .1), m, musx.vary(.3, .1), musx.vary(4, .1), reverb_amount=.1) #r*1.1
        t += r

This example uses the same instrument but doesn't sound anything like a fiddle!:

In [ ]:
# Constant morphing of pitch, timbre, amplitude, duration, location and reverb. 
def fm_morph(length, rhy, amp, cmratio=None):
    # increase probability of C4 G4 C5 until middle then decrease.
    DW = 0
    weights = [lambda: DW, lambda: DW, lambda: DW, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    probs = [0, 1, .2, 10, .3, 20, .35, 50, .45, 100, 
             .5, 100, .55, 100, .65, 25, .97, 20, .98, 10, 1, 1]
    notes = musx.hertz("c4 g4 c5 cs4 d4 ds4 e4 f4 fs4 gs4 a4 as4 b4")
    pitches = musx.Choose(notes, weights)
    ampenv = [0.0, 0, 0.02, 0.5, 0.03, 0.9, 0.05, 0.6, 0.75, 0.2, 1.0, 0]
    durenv = [0, .01, .2, .02, .3, .05, .35, .09, .4, .1, .425, .15, .45, .2, .5, 
              2.5, .55, .2, .575, .15, .6, .1, .65, .09, .7, .05, .8, .02, 1, .01]
    indenv = [0, 9, .2, 8, .3, 5, .35, 4, .45, 0, .5, 0, .65, 4, .96, 8, 1, 9]
    degree = [0,0, .5,45, 1,90]
    distance = [0,0, .5,0, 1,12]
    reverb = [0,.25, .5,.5, 1,.25]

    plotenvs([probs, ampenv, durenv, indenv, degree, distance, reverb], 
             titles=["prob", "amp", "dur", "index", "deg", "dist", "rev"], 
             figsize=(10,1), sharex=True, sharey=False)
    
    start = 0
    for i in range(length):
        # set current dynamic weight for C4 G4 C5
        DW = musx.interp(i, probs)
        ins.fm_violin(beg=start, dur=musx.interp(i/length, durenv),
                  frequency=pitches.next(), amplitude=amp,
                  fm_index=musx.interp(i/length, indenv), amp_env=ampenv,
                  degree=musx.interp(i/length, degree), 
                  distance=musx.interp(i/length, distance),
                  reverb_amount=musx.interp(i/length, reverb))
        start += rhy

with clm.Sound("test.wav", play=True, channels=2, reverb=ins.nrev(decay_time=2)):
    fm_morph(400, .1, .25)

In [ ]:
import pysndlib.instruments.fm_violin as FMG

##### More examples

Execute this %run statement to play a variety of interesting sounds composed by Bill Schottstaedt for his fm violin instrument. The script takes about 15-30 seconds to execute and produces a 3.5 minute stereo audio file ("./support/fmviolin.wav") containing over thirty sound examples, each separated by a few seconds of silence. Statistics about the audio file will be printed just before it plays:

In [ ]:
%run ./support/fmviolin.py

For more information on the fm_violin() see Schottstaedt's article [An Introduction To FM](https://ccrma.stanford.edu/software/snd/snd/fm.html) article and his JSTOR paper [The Simulation of Natural Instrument Tones using Frequency Modulation with a Complex Modulating Wave](https://www.jstor.org/stable/40731300).  

## Object oriented instruments

Up to now this notebook has generated audio files using function calls inside the `with Sound()` construct. It is also possible to generate  audio in an 'object oriented' manner using the standard Note, Score and Seq classes in musx. This next set of examples demonstrate how to create an 'audio note' object and use the musx Score, Seq and AudioFile classes to compose with it. 

This first step imports the musx.audio module and defines a new class to represent the pysndlib instrument. <!-- In this example the note class will be called GimpNote and it will wrap the gimp() function defined earlier in this notebook:In this example the call to musx.AudioNote() will translate the `gimp()` function into of musx.Note using .   -->  If no name for the new instrument class is provided the `AudioNote()` constructor will name the class using a pythonic version of the instrument's function name:

In [ ]:
import musx.audio as aud

print(gimp)

GimpNote = aud.AudioNote(gimp)

print(GimpNote)

Now that new GimpNote class has been defined we can create instances of it with  the object's attributes exactly corresponding to the function's parameter names and default values: 

In [ ]:
note = GimpNote(0, 1, 440, .3)
print("class:", note.__class__)
print("instance:", note)
print("note.start:", note.start)
print("note.dur:", note.dur)
print("note.freq:", note.freq)
print("note.amp:", note.amp)
print("note.degree:", note.degree)
print("note.parameters:", note.parameters())

Now define a composer function to generate GimpNotes:

In [ ]:
def gimper(score, rhy=.1, dur=.75, rate=1,  shift=1, amp=.75, reps=4, rev=0, dist=1):
    ampenv=musx.exp_env()
    tops = [440, 880, 440, 770, 440, 660, 440, 550, 440]
    bots = [220, 220, 220, 220, 220, 220, 220, 220, 220]
    for top, bot in zip(tops, bots):
        for time in musx.frange(0, reps, rhy):
            sinewave = math.sin(2*math.pi*rate*time) 
            degree = musx.rescale(sinewave, -1, 1, 0, 90)
            pitch = musx.rescale(sinewave, -1, 1, bot, top)
            note = GimpNote(score.now, dur, pitch*shift, amp, degree=degree, ampenv=ampenv,
                            reverb_amount=rev, distance=dist)
            score.add(note)
            yield rhy
            
print(f"instrument: {gimper}")

Next allocate and compose a score:

In [ ]:
score = musx.Score(out=musx.Seq())
score.compose(gimper(score))
print(score.out)
score.out.print(0,10)

To generate audio, allocate an Audiofile, pass it a file name and the sequence to render. Then use the file's `write()` method to generate the audio file, optionally passing `with Sound()` arguments to it except for the file name:

In [ ]:
file = aud.AudioFile("test.wav", score.out).write(play=True, statistics=True)
print(f"Wrote '{file.pathname}'")

## Optimizing a python instrument

This section explains how to increase the runtime speed of an audio instrument defined in the Python environment.  This notebook has defined four such instruments (simp(), gimp(), proto_fm(), and simple_fm()) the remaining insrumens (fm_violin())

Additive Synthesis produces a complex wave by summing together the phases, frequncies, amplitudes of its constituent partials. <!-- As the partials are layered togehter --> each partial is represented by an oscillor and amplitude envelope, so the more partials are included, the closer the sound comes to the original model, the more processing the software must do to render the sound.  At some point, an interpreted language like Pyton will struggle.

 each partial is represented by an oscillor and amplitude envelope, so the more partials are included, the closer the sound comes to the original model, and the more processing the software must do to render the sound.  

At some point an interpreted language like Pyton will struggle 



ll the audio examples in this notebook have been  fast and responsive. This is due to two reasons (1) the instruments themselves have been modest in the amount resources they require, (2) the

generating audio in an interpreted language like Python can be a difficult


Python is an interpreted language that, in some sense, "trades off" computational speed for expressibility and generating audio in Python is a challenging task due to the
The examples in this noteb


conventry bells additive synthsis instrument models aand then optimizes it to run at acceptable speeds.



In [ ]:
import support.coventry 
bellnames = [i for i in range(1, 11)]
tuning = support.coventry.coventry_primes
plain_bob = musx.Rotation(bellnames, [[0, 2, 1], [1, 2, 1]]).all(wrapped=True) 
numbells = len(plain_bob)
ldeg = [0, 45, numbells*.25, 45, numbells*.75, 0 ]
rdeg = [0, 45, numbells*.25, 45, numbells*.75, 90]
print(f"{numbells} bells in pattern, cranking...")
top, bot = bellnames[0],bellnames[-1]
hilite = [top, bot]

with clm.Sound(play=True, channels=2, reverb=ins.nrev(decay_time=6), statistics=True):
    beg = 0
    for i,b in enumerate(plain_bob):
        dur = musx.between(1.4, 3.8)
        amp = musx.between(.1,.15)
        support.coventry.bell(beg=beg, bell=b,
                 dur=dur if b not in hilite else dur * {top:1.5, bot: 2.25}[b],
                 freq=tuning[b],
                 amp=amp*1.1 if b not in hilite else amp * {top:1.8, bot:2.3}[b], #top:1.75, bot:2.35
                 deg=musx.between(musx.interp(i, ldeg), musx.interp(i, rdeg)),
                 dist=musx.interp(i, 0, 2,  numbells/2,  2,  numbells, 15),   
                 rev=musx.interp(i, 0, .001,numbells/2, .03, numbells, .06) )
        beg += musx.between(.27,.30)


In [ ]:
support.coventry.coventry_primes

Now create an instance of `SimpNote`: notice that its instance attributes exactly match the parameter names in the `simp()` function above. Since the function provides a default value for it's fourth parameter ('amp') the instance's 'amp' attribute is set to the same default value if it is not provided the constructor call:

In [ ]:

Code(filename='./support/coventry.py', language='python')

In [ ]:
print("foo")

In [ ]:
x = SimpNote(0, 2, 440)
print("x.start:", x.start)
print("x.dur:", x.dur)
print("x.freq:", x.freq)
print("x.amp", x.amp)

In [ ]:
def simple_fm (start, duration, carrier, amplitude, mc_ratio=1, fm_index=1,
               amp_env=[0,0,.25,1,.75,1,1,0], amp_attack=None, amp_release=None,
               index_env=[0, 1, 1, 1], index_attack=None, index_release=None,
               degree=45, distance=1, reverb=0):


In [ ]:
with clm.Sound("test.wav", play=True, channels=2):
    def foo(rate, shift=1, num=2, ratio=1):
        ampenv = musx.exp_env()
        amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
        index_env=amp_env

        
        rate = rate #1 # 2 3 7
        tops = [440, 880, 440, 770, 440, 660, 440, 550, 440]
        beg = 0
        for top in tops:
            location, frequency = [], []
            for time in musx.frange(0, num+.01, .1):
                sinewave = math.sin(2*math.pi*rate*time)
                degree = musx.rescale(sinewave, -1, 1, 0, 90)
                location.extend([time, degree])
                pitch = musx.rescale(sinewave, -1, 1, 220, top)
                frequency.extend([time, pitch])
                #simple_fm(beg+time, .75, pitch*shift, .75, degree=degree, amp_env=ampenv, mc_ratio=ratio)
                simple_fm(beg+time, .15, pitch*shift, .5, fm_index=2.0, amp_env=amp_env, index_env=index_env,
                          amp_attack=[1/3, .1], index_attack=[1/3, .1])
            beg += num+.01
    foo(1, 1/1, num=4, ratio=1)
        

In [ ]:
    amp_env=[0, 0, 1/12, 1, 1/3, .6, 5/6, .5, 1, 0]
    index_env=amp_env
    melody = musx.hertz( "g4 e c e g c5 e  d c e4 f# g g g e5 d c b4 a  b c5 c g4 e c")
    rhythm = musx.rhythm("e. s q*3   h  e. s q*3     h e*2 q. e q h  e. s q*5", tempo=80)
    t=0
    for m,r in zip(melody, rhythm):
        simple_fm(t, r, m, .5, fm_index=4.0, amp_env=amp_env, index_env=index_env,
                  amp_attack=[1/3, .1], index_attack=[1/3, .1])
        t += r

<instrument_table_data>

  <columns>
    <column Id="1" Name="Name"/>
    <column Id="2" Name="File"/>
    <column Id="3" Name="Category"/>
    <column Id="4" Name="Comment"/>
    <!-- stubbed out for now --> <column Id="5" Name="AutoLoad"/>
  </columns>

  <!-- Frequency Modulation -->

  <instruments>
    <instrument Name="animal" File="animals.scm" Category="Waveshaping" Comment="a bunch of animals" Examples="animals.clm" Depend="env.scm generators.scm" AutoLoad="no" />
    <instrument Name="anoi" File="anoi.scm" Category="Filtering" Comment="noise reduction" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="bigbird" File="bird.scm" Category="Waveshaping" Comment="waveshaping using polynomial unit generator" Examples="bird.clm" Depend="" AutoLoad="no" />
    <instrument Name="bird" File="bird.scm" Category="Waveshaping" Comment="sine wave instrument" Examples="bird.clm" Depend="" AutoLoad="no" />
    <instrument Name="bow" File="strad.scm" Category="Physical Modeling" Comment="Juan Reyes bowed string physical model" Examples="strad.clm" Depend="" AutoLoad="no" />
    <instrument Name="canter" File="bagpipes.scm" Category="Frequency Modulation" Comment="canter portion of Peter Commons' bagpipe (bag.clm)" Examples="bagpipes.clm" Depend="" AutoLoad="no" />
    <instrument Name="cellon" File="cellon.scm" Category="Frequency Modulation" Comment="feedback fm (Stanislaw Krupowiecz)" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="cnvrev" File="cnvrev.scm" Category="Sound Processing" Comment="Convolution of file with impulse" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="cross-fade" File="fade.scm" Category="Sound Processing" Comment="cross fade in the frequency domain" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="cut-samples" File="samples.scm" Category="Sound Processing" Comment="cut samples from file, see parse-samples" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="dlocsig" File="delocsig.scm" Category="Sound Processing" Comment="quad sound movement (Fernando Lopez-Lezcano)" Examples="" Depend="env.scm" AutoLoad="no" />
    <instrument Name="dissolve-fade" File="fade.scm" Category="Sound Processing" Comment="dissolving fade in the frequency domain" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="drone" File="bagpipes.scm" Category="Waveshaping" Comment="Peter Commons' bagpipe drone (bag.clm)" Examples="bagpipes.clm" Depend="" AutoLoad="no" />
    <instrument Name="env" File="env.scm" Category="Support" Comment="various envelope generators" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="expandn" File="expandn.scm" Category="Granular Synthesis" Comment="granular synthesis (Michael Klingbeil)" Examples="expandn.clm" Depend="env.scm" AutoLoad="no" />
    <instrument Name="expfil" File="expfil.scm" Category="Granular Synthesis" Comment="granulate two files interleaved" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="expsnd" File="expsnd.scm" Category="Granular Synthesis" Comment="granulate with envelopes on various fields" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="expsrc" File="expsrc.scm" Category="Granular Synthesis" Comment="granulate and src to mimic phase vocoder tricks" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="fm-bell" File="fm-bell.scm" Category="Frequency Modulation" Comment="fm bell (Michael McNabb)" Examples="fm-bell.clm" Depend="" AutoLoad="no" />
    <instrument Name="fm-drum" File="fm-drum.scm" Category="Frequency Modulation" Comment="cascade FM drum (Jan Mattox)" Examples="fm-drum.clm" Depend="" AutoLoad="no" />
    <instrument Name="fm-insect" File="fm-insect.scm" Category="Frequency Modulation" Comment="FM for insect simulations" Examples="fm-insect.clm" Depend="" AutoLoad="no" />
    <instrument Name="fm-noise" File="fm-noise.scm" Category="Frequency Modulation" Comment="FM for ocean wave simulations" Examples="" Depend="env.scm" AutoLoad="no" />
    <instrument Name="fm-trumpet" File="fm-trumpet.scm" Category="Frequency Modulation" Comment="Dexter Morrill's FM trumpet" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="fm-violin" File="v.scm" Category="Frequency Modulation" Comment="fm violin (William Schottstaedt)" Examples="v.clm" Depend="" AutoLoad="no" />
    <instrument Name="fm-voice" File="jcvoi.scm" Category="Frequency Modulation" Comment="John Chowning's fm voice" Examples="jcvoi.clm" Depend="env.scm" AutoLoad="no" />
    <instrument Name="fm" File="fm.scm" Category="Frequency Modulation" Comment="simple fm with envelope controls" Examples="fm.clm" Depend="" AutoLoad="no" />
    <instrument Name="fofins" File="fofins.scm" Category="Waveshaping" Comment="formant synthesis" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="freeverb" File="freeverb.scm" Category="Sound Processing" Comment="reverberation" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="fullmix" File="fullmix.scm" Category="Sound Processing" Comment="mixer" Examples="fullmix.clm" Depend="" AutoLoad="no" />
    <instrument Name="generator" File="generators.scm" Category="Support" Comment="various generators" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="gong" File="gong.scm" Category="Frequency Modulation" Comment="Paul Weineke's FM gong" Examples="gong.clm" Depend="env.scm" AutoLoad="no" />
    <instrument Name="gran-synth" File="gran-synth.scm" Category="Granular Synthesis" Comment="grains using wave-train gen" Examples="gran-synth.clm" Depend="" AutoLoad="no" />
    <instrument Name="grani" File="grani.scm" Category="Granular Synthesis" Comment="granular synthesis (Fernando Lopez-Lezcano)" Examples="" Depend="env.scm" AutoLoad="no" />
    <instrument Name="grapheq" File="grapheq.scm" Category="Filtering" Comment="graphic equalizer (Marco Trevisani)" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="hammondoid" File="hammondoid.scm" Category="Waveshaping" Comment="Perry Cook's Hammond organ (sort of)" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="jc-reverb" File="jcrev.scm" Category="Reverberation" Comment="John Chowning's ancient reverb" Examples="jcrev.clm" Depend="" AutoLoad="no" />
    <instrument Name="jl-reverb" File="jlrev.scm" Category="Reverberation" Comment="a more cavernous version of the same" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="lbj-piano" File="lbj-piano.scm" Category="Waveshaping" Comment="Doug Fulton's piano simulation using spectra of J.A.Moorer" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="maraca" File="maraca.scm" Category="Physical Modeling" Comment="maraca (Perry Cook)" Examples="maraca.clm" Depend="" AutoLoad="no" />
    <instrument Name="maxfilter" File="maxf.scm" Category="Filtering" Comment="Juan Reyes modal synthesis" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="metal" File="metal.scm" Category="Frequency Modulation" Comment="Perry Cook's heavy metal instrument" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="noise" File="noise.scm" Category="Frequency Modulation" Comment="Michael Scholz's FM noise instrument" Examples="" Depend="env.scm" AutoLoad="no" />
    <instrument Name="nrev" File="nrev.scm" Category="Reverberation" Comment="Michael McNabb's reverb" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="p" File="piano.scm" Category="Physical Modeling" Comment="Scott van Duyne's piano" Examples="" Depend="env.scm generators.scm" AutoLoad="no" />
    <instrument Name="p" File="piano.scm" Category="Physical Modeling" Comment="Scott van Duyne's piano" Examples="" Depend="env.scm generators.scm" AutoLoad="no" />
    <instrument Name="parse-samples" File="samples.scm" Category="Sound Processing" Comment="finds samples in an audio file, returns list of times" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="pins" File="pins.scm" Category="Filtering" Comment="spectral modeling a la SMS (Xavier Serra)" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="pluck" File="pluck.scm" Category="Physical Modeling" Comment="plucked string sound (David Jaffe)" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="pqw" File="pqw.scm" Category="Waveshaping" Comment="phase quadrature waveshaping" Examples="pqw.clm" Depend="" AutoLoad="no" />
    <instrument Name="pqw-vox" File="pqwvox.scm" Category="Waveshaping" Comment="like pqw but implements moving formant regions" Examples="pqwvox.clm" Depend="" AutoLoad="no" />
    <!-- <instrument Name="pvoc" File="pvoc.scm" Category="Filtering" Comment="phase vocoder (Michael Klingbeil and Michael Edwards)" Examples="" Depend="" AutoLoad="no" /> -->
    <instrument Name="resflt" File="resflt.scm" Category="Filtering" Comment="filter various kinds of input (Richard Karpen, Xavier Serra)" Examples="resflt.clm" Depend="" AutoLoad="no" />
    <instrument Name="reson" File="reson.scm" Category="Frequency Modulation" Comment="fm formants (John Chowning)" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="rhodey" File="rhodey.scm" Category="Frequency Modulation" Comment="Perry Cook's electric piano" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="scratch" File="scratch.scm" Category="Sound Processing" Comment="fancier version of backandforth" Examples="scratch.clm" Depend="" AutoLoad="no" />
    <instrument Name="singer" File="singer.scm" Category="Physical Modeling" Comment="spectral modelling" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="sndwarp" File="sndwarp.scm" Category="Sound Processing" Comment="Richard Karpen's SNDWARP instrument" Examples="" Depend="env.scm" AutoLoad="no" />
    <instrument Name="spectra" File="spectra.scm" Category="Waveshaping" Comment="wavetable output" Examples="spectra.clm" Depend="" AutoLoad="no" />
    <instrument Name="stereo-flute" File="stereo-flute.scm" Category="Physical Modeling" Comment="flute sound (Nicky Hind)" Examples="stereo-flute.clm" Depend="" AutoLoad="no" />  
    <instrument Name="stochastic" File="stochastic.scm" Category="Granular Synthesis" Comment="flute sound ()" Examples="" Depend="" AutoLoad="no" />  
    <instrument Name="touch-tone" File="touch-tone.scm" Category="Waveshaping" Comment="implement touch tone telephone sounds" Examples="touch-tone.clm" Depend="" AutoLoad="no" />
    <instrument Name="tubebell" File="tubebell.scm" Category="Frequency Modulation" Comment="Perry Cook's tubular bell" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="two-tab" File="two-tab.scm" Category="Waveshaping" Comment="interpolate between stored waveforms" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="vkey" File="vkey.scm" Category="Sound Processing" Comment="virtual sampler" Examples="" Depend="expandn.scm fullmix.scm" AutoLoad="no" />
    <instrument Name="vox" File="vox.scm" Category="Frequency Modulation" Comment="Mark LeBrun's fm voice instrument" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="wave" File="wave.scm" Category="Waveshaping" Comment="sinewave with envelope and gliss" Examples="wave.clm" Depend="" AutoLoad="no" />
    <instrument Name="sc:wave" File="wavesc.scm" Category="Waveshaping" Comment="Supercollider wave instrument" Examples="" Synthdef="wavesc.scd" Depend="" AutoLoad="no" />
    <instrument Name="wurley" File="wurley.scm" Category="Frequency Modulation" Comment="Perry Cook's Wurlitzer?" Examples="" Depend="" AutoLoad="no" />
    <instrument Name="za" File="za.scm" Category="Filtering" Comment="interpolating all pass filters" Examples="za.clm" Depend="" AutoLoad="no" />
    <instrument Name="zc" File="zc.scm" Category="Filtering" Comment="interpolating comb filters (phasing)" Examples="zc.clm" Depend="" AutoLoad="no" />
    <instrument Name="zipper" File="zip.scm" Category="Sound Processing" Comment="The 'digital zipper' effect." Examples="" Depend="" AutoLoad="no" />
    <instrument Name="zn" File="zn.scm" Category="Filtering" Comment="interpolating notch filters" Examples="zn.clm" Depend="" AutoLoad="no" />   
  </instruments>
</instrument_table_data>
